In [205]:
import pandas as pd
import numpy as np

Import Student Assessment Performance dataset, for now I am interested in the result of all students, so I will not be doing a breakdown by different demographics. This can change later on if necessary.

In [206]:
student_assessment_df = pd.read_csv('Student_Assessment_Performance.csv')
student_assessment_df.columns

Index(['School Year', 'District Code', 'District', 'School Code',
       'Organization', 'Assessment Name', 'ContentArea', 'Race', 'Gender',
       'Grade', 'SpecialDemo', 'Geography', 'SubGroup', 'RowStatus', 'Tested',
       'Proficient', 'PctProficient', 'ScaleScoreAvg'],
      dtype='object')

In [207]:
student_assessment_df = student_assessment_df[student_assessment_df['Race'] == 'All Students']
student_assessment_df = student_assessment_df[student_assessment_df['Gender'] == 'All Students']
student_assessment_df = student_assessment_df[student_assessment_df['SpecialDemo'] == 'All Students']
student_assessment_df = student_assessment_df[student_assessment_df['RowStatus'] == 'REPORTED']
student_assessment_df = student_assessment_df[student_assessment_df['Grade'] == 'All Students']
student_assessment_df.head()

,School Year,District Code,District,School Code,Organization,Assessment Name,ContentArea,Race,Gender,Grade,SpecialDemo,Geography,SubGroup,RowStatus,Tested,Proficient,PctProficient,ScaleScoreAvg
102,2015,0,State of Delaware,0,State of Delaware,Smarter Balanced Summative Assessment,MATH,All Students,All Students,All Students,All Students,All Students,All Students,REPORTED,67124.0,26016.0,38.76,NaN
239,2015,10,Caesar Rodney School District,0,Caesar Rodney School District,DCAS Alternate Assessment,ELA,All Students,All Students,All Students,All Students,All Students,All Students,REPORTED,141.0,80.0,56.74,NaN
379,2015,10,Caesar Rodney School District,0,Caesar Rodney School District,DCAS Alternate Assessment,MAT,All Students,All Students,All Students,All Students,All Students,All Students,REPORTED,141.0,71.0,50.35,NaN
447,2015,10,Caesar Rodney School District,0,Caesar Rodney School District,DCAS Alternate Assessment,SCI,All Students,All Students,All Students,All Students,All Students,All Students,REPORTED,41.0,23.0,56.10,NaN
503,2015,10,Caesar Rodney School District,0,Caesar Rodney School District,DCAS Alternate Assessment,SOC,All Students,All Students,All Students,All Students,All Students,All Students,REPORTED,57.0,18.0,31.58,NaN


In [208]:
student_assessment_df['ScaleScoreAvg'].mean(), student_assessment_df['ScaleScoreAvg'].std(), student_assessment_df['ScaleScoreAvg'].median(), \
    student_assessment_df['ScaleScoreAvg'].max(), student_assessment_df['ScaleScoreAvg'].min()

(1010.6339727105748, 933.6513346677887, 574.69, 2655.79, 7.8)

The column of interest for me is the PctProficient Column. This will be the target of my analysis. ScaleScoreAvg is another column measuring a similar thing I could consider but I decided against it for a few reasons. It is missing much more data, and it is unclear how the scores were derived. PctProficient comes from the 'Tested' and 'Proficient' columns, meaning those will correlate so I will get rid of them. Also all my demographic columns are now extraneous so I will delete them too.

In [209]:
student_assessment_df = student_assessment_df.drop(columns=['Race', 'Gender', 'SpecialDemo', 'RowStatus', 'Grade', 'SubGroup', 'Geography', 'ScaleScoreAvg','Tested','Proficient'])
student_assessment_df.columns

Index(['School Year', 'District Code', 'District', 'School Code',
       'Organization', 'Assessment Name', 'ContentArea', 'PctProficient'],
      dtype='object')

Now I will get the Enrollment information from the next dataset. This will give me the number of students enrolled at each school.

In [210]:
student_enrollment_df = pd.read_csv('Student_Enrollment.csv')
student_enrollment_df.columns

Index(['School Year', 'District Code', 'District', 'School Code',
       'Organization', 'Race', 'Gender', 'Grade', 'SpecialDemo', 'Geography',
       'SubGroup', 'RowStatus', 'Students', 'EOYEnrollment',
       'PctOfEOYEnrollment', 'FallEnrollment'],
      dtype='object')

I notice that the columns School Year, District Code, District, School Code, and Organization are carried over from the assessment dataframe so I will use those later in a merge. I want to count every student so I will get rid of the rows that count by demographics once again.

In [211]:
student_enrollment_df = student_enrollment_df[student_enrollment_df['Race'] == 'All Students']
student_enrollment_df = student_enrollment_df[student_enrollment_df['Gender'] == 'All Students']
student_enrollment_df = student_enrollment_df[student_enrollment_df['SpecialDemo'] == 'All Students']
student_enrollment_df = student_enrollment_df[student_enrollment_df['Grade'] == 'All Students']
student_enrollment_df.head()

,School Year,District Code,District,School Code,Organization,Race,Gender,Grade,SpecialDemo,Geography,SubGroup,RowStatus,Students,EOYEnrollment,PctOfEOYEnrollment,FallEnrollment
2034,2015,0,State of Delaware,0,State of Delaware,All Students,All Students,All Students,All Students,All Students,All Students,REDACTED,NaN,141336.0,NaN,134932.0
3543,2015,10,Caesar Rodney School District,0,Caesar Rodney School District,All Students,All Students,All Students,All Students,All Students,All Students,REDACTED,NaN,8333.0,NaN,7739.0
4013,2015,10,Caesar Rodney School District,610,Allen Frear Elementary School,All Students,All Students,All Students,All Students,All Students,All Students,REDACTED,NaN,746.0,NaN,699.0
4235,2015,10,Caesar Rodney School District,611,J. Ralph McIlvaine Early Childhood Center,All Students,All Students,All Students,All Students,All Students,All Students,REDACTED,NaN,642.0,NaN,551.0
4616,2015,10,Caesar Rodney School District,612,Major George S. Welch Elementary School,All Students,All Students,All Students,All Students,All Students,All Students,REDACTED,NaN,478.0,NaN,436.0


I will now get rid of the demographics columns as they are extraneous. The dataframe includes several measures of enrollment, taken at various times throughout the year. For now I am interested in the FallEnrollment, or enrollment on September 30th, which is the number that determines school funding in the state of Delaware.

In [212]:
student_enrollment_df = student_enrollment_df.drop(columns=['Race', 'Gender', 'SpecialDemo', 'RowStatus', 'SubGroup', 'Geography','EOYEnrollment', \
    'PctOfEOYEnrollment', 'Grade','Students'])
student_enrollment_df.columns

Index(['School Year', 'District Code', 'District', 'School Code',
       'Organization', 'FallEnrollment'],
      dtype='object')

I will now merge the the two dataframes based on the five columns mentioned earlier.

In [213]:
merged_df = student_assessment_df.merge(student_enrollment_df, on=['School Year', 'District Code', 'District','School Code', 'Organization'], how='inner')
merged_df.columns

Index(['School Year', 'District Code', 'District', 'School Code',
       'Organization', 'Assessment Name', 'ContentArea', 'PctProficient',
       'FallEnrollment'],
      dtype='object')

In addition to data on schools there are also aggregated rows on district and state level. For now I will get rid of them, I am only interested in schools.

In [214]:
merged_df['FallEnrollment'] = merged_df['FallEnrollment'].astype(float)
merged_df = merged_df[merged_df['Organization'].str.contains("State")==False]
merged_df = merged_df[merged_df['Organization'].str.contains("District")==False]

I realize I do not need quite so many indexes so I drop the district name and code columns.

In [215]:
merged_df = merged_df.drop(columns=['District Code', 'District'])
merged_df.columns

Index(['School Year', 'School Code', 'Organization', 'Assessment Name',
       'ContentArea', 'PctProficient', 'FallEnrollment'],
      dtype='object')

Now for the final dataset. This dataset includes information about the salaries, age, and experience of different jobs within the state. It also includes counts of each employee in the different jobs, this is what I am interested in.

In [216]:
educator_df = pd.read_csv('Educator_Average_Salary.csv')
educator_df.columns

Index(['School Year', 'District Code', 'District', 'School Code',
       'Organization', 'Race', 'Gender', 'Grade', 'SpecialDemo', 'Geography',
       'SubGroup', 'Staff Type', 'Staff Category', 'Job Classification',
       'Experience', 'Educators (FTE)', 'Average Total Salary',
       'Average State Salary', 'Average Local Salary',
       'Average Federal Salary', 'Average Years of Experience',
       'Average Years of Age'],
      dtype='object')

Once again I am interested in all employees and not interested in a demographic breakdown, so I get rid of those columns

In [217]:
educator_df = educator_df[educator_df['Race'] == 'All Educators']
educator_df = educator_df[educator_df['Gender'] == 'All Educators']
educator_df = educator_df[educator_df['SpecialDemo'] == 'All Educators']
educator_df = educator_df[educator_df['Experience'] == 'ALL']
educator_df = educator_df[educator_df['Organization'].str.contains("State")==False]
educator_df = educator_df[educator_df['Organization'].str.contains("District")==False]

The column I am interested in is job classification, this is a list of all the different jobs in a school, here I am interested in the categories so I will get rid of the row for all jobs.

In [218]:
educator_df = educator_df[['School Year', 'School Code', 'Organization', 'Job Classification', 'Educators (FTE)']]
educator_df = educator_df[educator_df['Job Classification'] != 'ALL']

I create a list of all jobs, for later

In [219]:
listofjobs = educator_df['Job Classification'].unique()
listofjobs

array(['Specialist, Instructional', 'Assistant Superintendent',
       'Administrative Assistant', 'Supervisor, Pupil Support',
       'Crafts & Trades', 'Technical', 'Supervisor, General Support',
       'Custodial', 'Managerial', 'Director', 'Supervisor, Instructional',
       'Secretarial', 'Superintendent', 'Cafeteria Worker',
       'Specialist, General Support', 'Home Visitor',
       'Other Pupil Support', 'Psychologist',
       'Teacher, Special Elementary', 'Speech and Hearing Therapist',
       'Other Instructional Support', 'Specialist, Pupil Support',
       'Teaching & Clerical Aide', 'Teacher, Special Secondary', 'Nurse',
       'Teacher, Regular Secondary', 'Social Worker', 'Principal',
       'Teacher, Regular Elementary', 'Guidance Counselor',
       'Assistant Principal', 'Librarian', 'Service Aide',
       'Bus Driver & Laborer', 'Other General Support', 'Psychometrist'],
      dtype=object)

I see that the jobs for teachers are broken down into elementary and secondary classifications. However this is unnecessary because few secondary schools have elementary teachers and vice versa. It could also confuse a model and cause suggestions like elementary schools should hire more high school teachers, and absurdity. I will combine these columns.

In [220]:
educator_df = pd.pivot(educator_df, index=['School Year', 'School Code', 'Organization'], columns='Job Classification', values='Educators (FTE)')
educator_df = educator_df.reset_index()
educator_df.columns

Index(['School Year', 'School Code', 'Organization',
       'Administrative Assistant', 'Assistant Principal',
       'Assistant Superintendent', 'Bus Driver & Laborer', 'Cafeteria Worker',
       'Crafts & Trades', 'Custodial', 'Director', 'Guidance Counselor',
       'Home Visitor', 'Librarian', 'Managerial', 'Nurse',
       'Other General Support', 'Other Instructional Support',
       'Other Pupil Support', 'Principal', 'Psychologist', 'Psychometrist',
       'Secretarial', 'Service Aide', 'Social Worker',
       'Specialist, General Support', 'Specialist, Instructional',
       'Specialist, Pupil Support', 'Speech and Hearing Therapist',
       'Superintendent', 'Supervisor, General Support',
       'Supervisor, Instructional', 'Supervisor, Pupil Support',
       'Teacher, Regular Elementary', 'Teacher, Regular Secondary',
       'Teacher, Special Elementary', 'Teacher, Special Secondary',
       'Teaching & Clerical Aide', 'Technical'],
      dtype='object', name='Job Classificat

Fill all NaN rows in teacher count columns with 0, so that the addition works

In [221]:
educator_df['Teacher, Regular Elementary'] = educator_df['Teacher, Regular Elementary'].fillna(0)
educator_df['Teacher, Special Elementary'] = educator_df['Teacher, Special Elementary'].fillna(0)
educator_df['Teacher, Regular Secondary'] = educator_df['Teacher, Regular Secondary'].fillna(0)
educator_df['Teacher, Special Secondary'] = educator_df['Teacher, Special Secondary'].fillna(0)

In [222]:
educator_df['Teacher, Regular'] = educator_df['Teacher, Regular Elementary'] + educator_df['Teacher, Regular Secondary']
educator_df['Teacher, Special'] = educator_df['Teacher, Special Elementary'] + educator_df['Teacher, Special Secondary']

Now merge educator_df with the earlier merged_df on the school code, year, and organization columns

In [223]:
df = merged_df.merge(educator_df, on=['School Year', 'School Code', 'Organization'], how='inner')
df.head()

,School Year,School Code,Organization,Assessment Name,ContentArea,PctProficient,FallEnrollment,Administrative Assistant,Assistant Principal,Assistant Superintendent,...,"Supervisor, Instructional","Supervisor, Pupil Support","Teacher, Regular Elementary","Teacher, Regular Secondary","Teacher, Special Elementary","Teacher, Special Secondary",Teaching & Clerical Aide,Technical,"Teacher, Regular","Teacher, Special"
0,2015,610,Allen Frear Elementary School,Smarter Balanced Summative Assessment,ELA,69.06,699.0,NaN,NaN,NaN,...,NaN,NaN,35.0,0.0,6.0,0.0,6.0,NaN,35.0,6.0
1,2015,610,Allen Frear Elementary School,Smarter Balanced Summative Assessment,MATH,59.75,699.0,NaN,NaN,NaN,...,NaN,NaN,35.0,0.0,6.0,0.0,6.0,NaN,35.0,6.0
2,2015,612,Major George S. Welch Elementary School,Smarter Balanced Summative Assessment,ELA,80.84,436.0,NaN,1.0,NaN,...,NaN,NaN,26.0,0.0,3.0,0.0,5.0,NaN,26.0,3.0
3,2015,612,Major George S. Welch Elementary School,Smarter Balanced Summative Assessment,MATH,65.12,436.0,NaN,1.0,NaN,...,NaN,NaN,26.0,0.0,3.0,0.0,5.0,NaN,26.0,3.0
4,2015,616,Nellie Hughes Stokes Elementary School,Smarter Balanced Summative Assessment,ELA,63.08,442.0,NaN,1.0,NaN,...,NaN,NaN,25.0,0.0,8.0,0.0,7.0,NaN,25.0,8.0


Now find the students per teacher columns

In [224]:
df['Students Per Teacher, Regular'] = df['FallEnrollment'] / df['Teacher, Regular']
df['Students Per Teacher, Special'] = df['FallEnrollment'] / df['Teacher, Special']

del merged_df
del student_assessment_df
del student_enrollment_df
del educator_df

Now I will do the same operation and find the students per staff of all types of staff, additionally I will drop the normal count columns

In [225]:
for item in listofjobs:
    df['Students Per ' + str(item)] = df['FallEnrollment'] / df[item]
    df = df.drop(item, axis=1)

The earlier division has the possibility of division by zero. I will deal with this by changing the infinite cells to zero.

In [226]:
df = df.drop(['Teacher, Regular','Teacher, Special','Students Per Teacher, Regular Elementary', 'Students Per Teacher, Special Elementary', \
    'Students Per Teacher, Regular Secondary', 'Students Per Teacher, Special Secondary'], axis=1)
df = df.replace([np.inf, -np.inf], np.nan)

df.shape

(4479, 41)

Now view the final dataframe and write it to a file.

In [204]:
df.to_csv('df.csv', index=False)